# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**: Regalado Floriano Luis A.

**Professor**: Pablo Camarillo Ramirez

# Machine Learning algorithm to use

Since we will be using, https://www.kaggle.com/datasets/mohankrishnathalla/global-house-purchase-decision-dataset , the most appropriate machine learning algorithm to use would be that of Support Vector Machines, since we would like to predict whether a house will be bought or not from factors such as the region and the kind of house. 


# Dataset

The Dataset is https://www.kaggle.com/datasets/mohankrishnathalla/global-house-purchase-decision-dataset

Size of the dataset:
    The Dataset is composed of  200,000  entries, with 25 columns each


As it will be seen, the dataset is balanced in all of it's country, bar each houses's price, as well as it's city, which is to be expected.

Nonetheless, cities tend to be balanced within their own country.

We will  be training the model on the city the house is in, it's type, furbishing status, size, attributes, legal status, and, of course, price.

### Load

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Logistic Regression") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/work-dir/jars/postgresql-42.7.8.jar") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

25/11/22 01:26:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from regalado_floriano.spark_utils import SparkUtils 

In [3]:
houses_schema = SparkUtils.generate_schema(
 
    (("property_id","int"),
("country","string"),
("city","string"),
("property_type","string"),
("furnishing_status","string"),
("property_size_sqft","int"),
("price","int"),
("constructed_year","int"),
("previous_owners","int"),
("rooms","int"),
("bathrooms","int"),
("garage","bool"),
("garden","bool"),
("crime_cases_reported","int"),
("legal_cases_on_property","bool"),
("customer_salary","int"),
("loan_amount","int"),
("loan_tenure_years","int"),
("monthly_expenses","int"),
("down_payment","int"),
("emi_to_income_ratio","float"),
("satisfaction_score","int"),
("neighbourhood_rating","int"),
("connectivity_score","int"),
("decision","bool")
    )
)

In [4]:
house_df = spark.read \
                .option("header", "true") \
                .schema(houses_schema) \
                .csv("/opt/spark/work-dir/data/house_purchases")
house_df= house_df.na.fill({"country":"unknown"})
house_df= house_df.na.fill(False)

In [5]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.sql.functions import when, col

In [6]:
country_count = house_df.groupBy("country").count()

In [7]:
countries = "France,USA,Singapore,Japan,Canada,Australia,Germany,Brazil,UK,South Africa,UAE,China,India".split(",")

In [8]:
for country in countries:
    toGroup = house_df.filter(col("country")==country)
    grouped_country = toGroup.groupBy("city").count() 
    grouped_country.describe().show()

+-------+-----+------------------+
|summary| city|             count|
+-------+-----+------------------+
|  count|    3|                 3|
|   mean| NULL| 5209.333333333333|
| stddev| NULL|124.34360994169879|
|    min| Lyon|              5080|
|    max|Paris|              5328|
+-------+-----+------------------+

+-------+-------------+-----------------+
|summary|         city|            count|
+-------+-------------+-----------------+
|  count|            5|                5|
|   mean|         NULL|           3056.2|
| stddev|         NULL|52.46617958266067|
|    min|      Chicago|             2999|
|    max|San Francisco|             3107|
+-------+-------------+-----------------+

+-------+---------+-------+
|summary|     city|  count|
+-------+---------+-------+
|  count|        1|      1|
|   mean|     NULL|15278.0|
| stddev|     NULL|   NULL|
|    min|Singapore|  15278|
|    max|Singapore|  15278|
+-------+---------+-------+

+-------+-----+-----------------+
|summary| city|   

In [9]:
categories = "country city property_type furnishing_status".split()


In [10]:
for cat in categories:
    to_describe = house_df.groupBy(cat).count()
    to_describe.describe().show()

+-------+---------+------------------+
|summary|  country|             count|
+-------+---------+------------------+
|  count|       13|                13|
|   mean|     NULL|15384.615384615385|
| stddev|     NULL|120.51109109506515|
|    min|Australia|             15141|
|    max|      USA|             15628|
+-------+---------+------------------+

+-------+---------+------------------+
|summary|     city|             count|
+-------+---------+------------------+
|  count|       40|                40|
|   mean|     NULL|            5000.0|
| stddev|     NULL|2312.3473587876692|
|    min|Abu Dhabi|              2510|
|    max|Vancouver|             15278|
+-------+---------+------------------+

+-------+-------------+------------------+
|summary|property_type|             count|
+-------+-------------+------------------+
|  count|            6|                 6|
|   mean|         NULL|33333.333333333336|
| stddev|         NULL| 172.1135284243126|
|    min|    Apartment|             33

# ML Training process

We will  be training the model on the city the house is in, it's type, furbishing status, size, attributes, legal status, and, of course, price.
However, many of those values are strings. We will therefore be encoding each of these as ints.

In [11]:
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id
house_df= house_df.na.fill(False)


In [12]:
features="country,city,property_type,furnishing_status,property_size_sqft,price,constructed_year,previous_owners,rooms,bathrooms,garage,garden,crime_cases_reported,legal_cases_on_property,neighbourhood_rating".split(",")
categorical_features="country,city,property_type,furnishing_status".split(",")

## Transformation 1: Creating unique ids

In [13]:
#in order to prepare for ease of analysis, all categorical data will be put in it's own frame. That is to say, each country and city will get 
# their own id
_localMap = SparkUtils.generate_keyed_distinct_column(house_df)

categoricalTables =   {
     key:     _localMap(key) for key in categorical_features
 }

id_house = house_df
for cat in categorical_features:
    cur_df = categoricalTables[cat]
    id_house = SparkUtils.replace_column_for_key(id_house)(cur_df)(cat)


In [14]:
id_house.printSchema()

root
 |-- property_id: integer (nullable = true)
 |-- property_size_sqft: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- constructed_year: integer (nullable = true)
 |-- previous_owners: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- garage: boolean (nullable = false)
 |-- garden: boolean (nullable = false)
 |-- crime_cases_reported: integer (nullable = true)
 |-- legal_cases_on_property: boolean (nullable = false)
 |-- customer_salary: integer (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- loan_tenure_years: integer (nullable = true)
 |-- monthly_expenses: integer (nullable = true)
 |-- down_payment: integer (nullable = true)
 |-- emi_to_income_ratio: float (nullable = true)
 |-- satisfaction_score: integer (nullable = true)
 |-- neighbourhood_rating: integer (nullable = true)
 |-- connectivity_score: integer (nullable = true)
 |-- decision: boolean (nullable = false)
 |-- country

In [15]:
 
for cat in categorical_features:
    cur_df = categoricalTables[cat]
    cur_df.printSchema()
    cur_df.groupBy(cat).count().describe().show()


root
 |-- country: string (nullable = false)
 |-- id: long (nullable = false)

+-------+---------+-----+
|summary|  country|count|
+-------+---------+-----+
|  count|       13|   13|
|   mean|     NULL|  1.0|
| stddev|     NULL|  0.0|
|    min|Australia|    1|
|    max|      USA|    1|
+-------+---------+-----+

root
 |-- city: string (nullable = true)
 |-- id: long (nullable = false)

+-------+---------+-----+
|summary|     city|count|
+-------+---------+-----+
|  count|       40|   40|
|   mean|     NULL|  1.0|
| stddev|     NULL|  0.0|
|    min|Abu Dhabi|    1|
|    max|Vancouver|    1|
+-------+---------+-----+

root
 |-- property_type: string (nullable = true)
 |-- id: long (nullable = false)

+-------+-------------+-----+
|summary|property_type|count|
+-------+-------------+-----+
|  count|            6|    6|
|   mean|         NULL|  1.0|
| stddev|         NULL|  0.0|
|    min|    Apartment|    1|
|    max|        Villa|    1|
+-------+-------------+-----+

root
 |-- furnishing_

In [16]:

locations = id_house.select( "country_id", "city_id").distinct().withColumn("id",monotonically_increasing_id())
id_house = id_house.join(
    locations ,
    on=[id_house["country_id"] == locations["country_id"],
        id_house["city_id"] == locations["city_id"]],
    how="left"
).drop("country_id", "city_id").withColumnRenamed("id", "location_id")


25/11/22 01:27:44 WARN Column: Constructing trivially true equals predicate, 'm.country_id == m.country_id'. Perhaps you need to use aliases.
25/11/22 01:27:44 WARN Column: Constructing trivially true equals predicate, 'm.city_id == m.city_id'. Perhaps you need to use aliases.


In [17]:
id_house.printSchema()

root
 |-- property_id: integer (nullable = true)
 |-- property_size_sqft: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- constructed_year: integer (nullable = true)
 |-- previous_owners: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- garage: boolean (nullable = false)
 |-- garden: boolean (nullable = false)
 |-- crime_cases_reported: integer (nullable = true)
 |-- legal_cases_on_property: boolean (nullable = false)
 |-- customer_salary: integer (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- loan_tenure_years: integer (nullable = true)
 |-- monthly_expenses: integer (nullable = true)
 |-- down_payment: integer (nullable = true)
 |-- emi_to_income_ratio: float (nullable = true)
 |-- satisfaction_score: integer (nullable = true)
 |-- neighbourhood_rating: integer (nullable = true)
 |-- connectivity_score: integer (nullable = true)
 |-- decision: boolean (nullable = false)
 |-- propert

## Transformation 3: Final prep

In [18]:
non_cat = list(set(features) - set(categorical_features))
train_features = ["location_id"]
train_features.extend(non_cat)
train_features.extend([f"{x}_id" for x in categorical_features if x not in ("country,city").split(",") ])
to_select = ["decision"]
to_select.extend(train_features)
to_train = id_house.select(to_select)
to_train = to_train.withColumn("decision", to_train.decision.cast("float"))
to_train.printSchema()

root
 |-- decision: float (nullable = false)
 |-- location_id: long (nullable = true)
 |-- garden: boolean (nullable = false)
 |-- rooms: integer (nullable = true)
 |-- previous_owners: integer (nullable = true)
 |-- legal_cases_on_property: boolean (nullable = false)
 |-- property_size_sqft: integer (nullable = true)
 |-- crime_cases_reported: integer (nullable = true)
 |-- neighbourhood_rating: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- garage: boolean (nullable = false)
 |-- constructed_year: integer (nullable = true)
 |-- property_type_id: long (nullable = true)
 |-- furnishing_status_id: long (nullable = true)



## Transformation 4: Assembling columns

In [19]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
stringIndexer = StringIndexer(inputCol="decision", outputCol="indexed_decision",
stringOrderType="frequencyDesc")

assembler = VectorAssembler(inputCols=features, outputCol="features")


In [20]:

assembler = VectorAssembler(inputCols=train_features, outputCol="features")

data_with_features = assembler.transform(to_train).select("decision", "features")# Step 2: Scale the features vector
scaler = StandardScaler(
inputCol="features",
outputCol="scaledFeatures",
withMean=True,
# centers the data
withStd=True
# scales to unit std
)
scaler_model = scaler.fit(data_with_features)
scaled_data = scaler_model.transform(data_with_features)

25/11/22 01:28:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [21]:
train_df, test_df = scaled_data.randomSplit([0.8, 0.2], seed=4) #seed chosen at random by a fair dice

We will be testing on a reduced dataset (20% of the full size) and testing the predictions on the rest of it, as per the slides.

## Model Creation

In [24]:
from pyspark.ml.classification import LinearSVC, OneVsRest

# Initialize the LinearSVC classifier for binary classification
lsvc = LinearSVC(maxIter=10, regParam=0.01, labelCol="decision", predictionCol="prediction")

# Set up OneVsRest classifier for multi-class classification
ovr = OneVsRest(classifier=lsvc,  labelCol="decision", predictionCol="prediction")

In [25]:
ovr_model = ovr.fit(train_df)# Make predictions on the test set
predictions = ovr_model.transform(test_df)

In [26]:
ova_model_path = "/opt/spark/work-dir/data/mlmodels/svm/svm_proy"
ovr_model.write().overwrite().save(ova_model_path)

In [27]:
from pyspark.ml.classification import OneVsRestModel

# Retreive the saved model
ovr_model = OneVsRestModel.load(ova_model_path)

# Use the trained model to make predictions on the test data
predictions = ovr_model.transform(test_df)

# Show predictions
predictions.select("scaledFeatures", "prediction").show(truncate=False)

[Stage 257:>                                                        (0 + 1) / 1]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|scaledFeatures                                                                                                                                                                                                                          |prediction|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|[-1.5694208460689911,0.0,-1.5306455719218193,-0.9995862879360372,0.0,-1.648544423938515,-1.0368917164501572,0.5206509112794548,-0.9560395756378267,-1.08796540066734,0.0,0.1899112096173136,-1.4685206377004068,-1.222746414315009]     |0.0       |
|[-1.56942084606

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="decision",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})

f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")  

[Stage 324:============================>                            (1 + 1) / 2]

Accuracy: 1.0
F1 Score: 1.0
